In [1]:
import time
import torch
from torch import nn,optim
import torchvision
import torchvision.transforms as transforms

In [2]:
device=torch.device('cpu')

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5),  # in channel,out channel,kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2))
        self.fc = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120), 
            nn.Sigmoid(),                    
            nn.Linear(120, 84), 
            nn.Sigmoid(),
            nn.Linear(84, 10))
    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

In [4]:
net=LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [5]:
isinstance(net, torch.nn.Module)

True

In [23]:
def evaluate_accuracy(data_iter, net, device=torch.device('cpu')):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(x.to(device)).argmax(
                    dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()
            else:
                if ('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(x, istraining=False).argmax(
                        dim=1) == y).float().sum().item()
                else:
                    acc_sum += (net(x).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

In [27]:
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device,
              num_epochs):
    net = net.to(device)
    print('training on ', device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for x, y, in train_iter:
            x = x.to(device)
            y = y.to(device)
            y_hat = net(x)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(
            'epoch %d ,loss %.4f,train acc %.3f,test acc %.3f,time %.1f sec' %
            (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc,
             time.time() - start))

In [28]:
batch_size=256
mnist_train=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=True,
    download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=False,
    download=True,transform=transforms.ToTensor())
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,
                                      shuffle=True,num_workers=4)

test_iter=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,
                                      shuffle=True,num_workers=4)

In [31]:
lr,num_epochs=0.001,5
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cpu
epoch 1 ,loss 0.3593,train acc 0.866,test acc 0.851,time 21.9 sec
epoch 2 ,loss 0.1765,train acc 0.869,test acc 0.855,time 26.2 sec
epoch 3 ,loss 0.1167,train acc 0.871,test acc 0.860,time 22.5 sec
epoch 4 ,loss 0.0863,train acc 0.872,test acc 0.861,time 22.3 sec
epoch 5 ,loss 0.0680,train acc 0.874,test acc 0.862,time 23.0 sec
